In [104]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import gzip
from scipy.io import mmread
import matplotlib.pyplot as plt
from bitstring import BitStream
import huffman

In [ ]:
def load_data(dataset):
  datasets = ['humanBloodCells', 'humanKidneyCells', 'humanSkinCells']
  dataset_index = datasets.index(dataset)
  with gzip.open('./data/' + datasets[dataset_index] + '/matrix.mtx.gz', 'rt') as f:
    matrix = np.array(mmread(f).todense())
    features = pd.read_csv('./data/' + datasets[dataset_index] + '/features.tsv.gz', sep='\t')
    barcodes = pd.read_csv('./data/' + datasets[dataset_index] + '/barcodes.tsv.gz', sep='\t')
  return matrix, features, barcodes

In [ ]:
def print_stats(matrix):
  num_nonzero = np.count_nonzero(matrix)
  sparsity = np.count_nonzero(matrix) / matrix.size
  print('Number of non-zero entries:', num_nonzero)
  print('Matrix shape:', matrix.shape)
  print('Sparsity:', f'{sparsity * 100}%')

In [107]:
def plot_non_zero_frequencies(matrix, matrix_name):
  non_zero_vals = [i for i in matrix.reshape(matrix.size).tolist() if i != 0]
  plt.xlabel('Non-zero values')
  plt.ylabel('Frequency')
  plt.title(f'Non-zero value frequencies of {matrix_name}')
  plt.hist(non_zero_vals, bins=100, log=True)

In [131]:
def kmeans(matrix, n_clusters):
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(matrix)

  # Identify cluster representatives
  min_dists = np.zeros(n_clusters)
  reps = np.array([-1 for i in range(n_clusters)])
  for i in range(matrix.shape[0]):
    curr_dist = np.linalg.norm(matrix[i] - kmeans.cluster_centers_[kmeans.labels_[i]])
    if reps[kmeans.labels_[i]] == -1 or curr_dist < min_dists[kmeans.labels_[i]]:
      min_dists[kmeans.labels_[i]] = curr_dist
      reps[kmeans.labels_[i]] = i

  # Get delta matrix
  delta_matrix = np.copy(matrix)
  for i in range(matrix.shape[0]):
    if reps[kmeans.labels_[i]] == i:
      continue
    delta_matrix[i] -= matrix[reps[kmeans.labels_[i]]]

  return reps, kmeans.labels_, delta_matrix

In [132]:
SIZE_BITS = 32
METADATA_BITS = 8

def compress_delta_matrix(delta_matrix, bitstream, bits_per_n_index):
  for i in range(delta_matrix.shape[0]):
    row = delta_matrix[i]
    num_nonzero = np.count_nonzero(row)
    bits_per_val = int(np.max(np.abs(row))).bit_length()
    bitstream.append(f'bin{bits_per_n_index}={bin(num_nonzero)[2:].zfill(bits_per_n_index)}')
    bitstream.append(f'uint{METADATA_BITS}={bits_per_val}')
    for j in range(delta_matrix.shape[1]):
      if row[j] == 0:
        continue
      bitstream.append(f'bin{bits_per_n_index}={bin(j)[2:].zfill(bits_per_n_index)}')
      bitstream.append('0b0' if row[j] > 0 else '0b1')
      bitstream.append(f'bin{bits_per_val}={bin(abs(row[j]))[2:].zfill(bits_per_val)}')

def decompress_delta_matrix(delta_matrix, bitstream, bits_per_n_index):
  for i in range(delta_matrix.shape[0]):
    row = delta_matrix[i]
    num_nonzero = int(bitstream.read(f'bin{bits_per_n_index}'), 2)
    bits_per_val = bitstream.read(f'uint{METADATA_BITS}')
    for j in range(num_nonzero):
      index = int(bitstream.read(f'bin{bits_per_n_index}'), 2)
      sign = 1 if bitstream.read(1) == '0b0' else -1
      value = int(bitstream.read(f'bin{bits_per_val}'), 2)
      row[index] = sign * value

In [133]:
# Huffman Encoding functions
def get_huffman_tree(freqs):
  freqs = [(int(k), v) for k, v in freqs.items()]
  huffman_tree = huffman.Tree(freqs)
  return huffman_tree

def encode_huffman_tree(root, bits_per_val, bitarray):
  if type(root) == huffman.Leaf:
    bitarray.append('0b1')
    bitarray.append('0b1' if root.symbol < 0 else '0b0')
    bitarray.append(f'0b{bin(abs(root.symbol))[2:].zfill(bits_per_val)}')
    return
  bitarray.append('0b0')
  encode_huffman_tree(root.left, bits_per_val, bitarray)
  encode_huffman_tree(root.right, bits_per_val, bitarray)

def decode_huffman_tree(bitstream, bits_per_val):
  if bitstream.read(1) == '0b1':
    mult = -1 if bitstream.read(1) == '0b1' else 1
    val = mult * int(bitstream.read(f'bin{bits_per_val}'), 2)
    return huffman.Leaf(val, 0)
  left = decode_huffman_tree(bitstream, bits_per_val)
  right = decode_huffman_tree(bitstream, bits_per_val)
  return huffman.Node(left, right)

def read_huffman_tree(root, bitstream):
  if type(root) == huffman.Leaf:
    return root.symbol
  elif bitstream.read(1) == '0b0':
    return read_huffman_tree(root.left, bitstream)
  return read_huffman_tree(root.right, bitstream)

def compress_delta_matrix_huffman(delta_matrix, bitstream, bits_per_n_index, mode="vals_only"):
  freqs = {}
  bits_per_huffman_val = 0
  for i in range(delta_matrix.shape[0]):
    for j in range(delta_matrix.shape[1]):
      if delta_matrix[i][j] != 0:
        if delta_matrix[i][j] not in freqs:
          bits_per_huffman_val = max(bits_per_huffman_val, int(delta_matrix[i][j]).bit_length())
          freqs[delta_matrix[i][j]] = 0
        freqs[delta_matrix[i][j]] += 1

        if mode == "vals_indices":
          if j not in freqs:
            bits_per_huffman_val = max(bits_per_huffman_val, j.bit_length())
            freqs[j] = 0
          freqs[j] += 1

  huffman_tree = get_huffman_tree(freqs)
  huffman_codebook = huffman_tree.codebook
  bitstream.append(f'uint{METADATA_BITS}={bits_per_huffman_val}')  
  encode_huffman_tree(huffman_tree.root, bits_per_huffman_val, bitstream)

  for i in range(delta_matrix.shape[0]):
    row = delta_matrix[i]
    num_nonzero = np.count_nonzero(row)
    bitstream.append(f'bin{bits_per_n_index}={bin(num_nonzero)[2:].zfill(bits_per_n_index)}')
    for j in range(delta_matrix.shape[1]):
      if row[j] == 0:
        continue
      if mode == "vals_only":
        bitstream.append(f'bin{bits_per_n_index}={bin(j)[2:].zfill(bits_per_n_index)}')
      elif mode == "vals_indices":
        bitstream.append(f'0b{huffman_codebook[j]}')
      bitstream.append(f'0b{huffman_codebook[row[j]]}')

def decompress_delta_matrix_huffman(delta_matrix, bitstream, bits_per_n_index, mode="vals_only"):
  huffman_tree = decode_huffman_tree(bitstream, bitstream.read(f'uint{METADATA_BITS}'))
  for i in range(delta_matrix.shape[0]):
    num_nonzero = int(bitstream.read(f'bin{bits_per_n_index}'), 2)
    for j in range(num_nonzero):
      index = None
      if mode == "vals_only":
        index = int(bitstream.read(f'bin{bits_per_n_index}'), 2)
      elif mode == "vals_indices":
        index = read_huffman_tree(huffman_tree, bitstream)
      val = read_huffman_tree(huffman_tree, bitstream)
      delta_matrix[i][index] = val


In [189]:
# RLE functions
def get_run_lengths(bitstream):
  bitstream.pos = 0
  run_lengths = []
  curr_bit = -1
  run_len = 0
  initial_bit = -1
  for i in range(bitstream.length):
    bit = 1 if bitstream.read(1) == '0b1' else 0
    if bit == curr_bit:
      run_len += 1
    else:
      if curr_bit == -1:
        initial_bit = bit
      curr_bit = bit
      run_lengths.append(run_len)
      run_len = 1
  run_lengths.append(run_len)
  del run_lengths[0]
  return initial_bit, run_lengths

def encode_rle(bitstream, mode="elias", runs=None):
  initial_bit = None
  run_lengths = None
  if runs != None:
    (initial_bit, run_lengths) = runs
  else:
    initial_bit, run_lengths = get_run_lengths(bitstream)
  rle_bitstream = BitStream()
  if mode == "elias":
    for run_len in run_lengths:
      rle_bitstream.append(f'0b{initial_bit}')
      rle_bitstream.append(f'0b{bin(run_len)[2:].zfill(run_len.bit_length() * 2 - 1)}')
  elif mode == "huffman":
    run_length_freqs = {}
    bits_per_run_length = 0
    for run_len in run_lengths:
      if run_len not in run_length_freqs:
        bits_per_run_length = max(bits_per_run_length, run_len.bit_length())
        run_length_freqs[run_len] = 0
      run_length_freqs[run_len] += 1
    huffman_tree = get_huffman_tree(run_length_freqs)
    huffman_codebook = huffman_tree.codebook
    print('Huffman codebook:', huffman_codebook)
    rle_bitstream.append(f'uint{METADATA_BITS}={bits_per_run_length}')  
    encode_huffman_tree(huffman_tree.root, bits_per_run_length, rle_bitstream)

    rle_bitstream.append(f'0b{initial_bit}')
    for run_len in run_lengths:
      rle_bitstream.append(f'0b{huffman_codebook[run_len]}')
  return rle_bitstream  

def decode_rle(rle_bitstream, mode="elias"):
  bitstream = BitStream()
  if mode == "elias":
    curr_bit = 1 if rle_bitstream.read(1) == '0b1' else 0
    num_zeros = 0
    for i in range(rle_bitstream.length - 1):
      bit = 1 if rle_bitstream.read(1) == '0b1' else 0
      if bit == 0:
        num_zeros += 1
      else:
        run_len = int('1' + bitstream.read(f'bin{num_zeros}'), 2)
        for j in range(run_len):
          bitstream.append(f'0b{curr_bit}')
        num_zeros = 0
        curr_bit = 1 - curr_bit
    run_len = int('1' + bitstream.read(f'bin{num_zeros}'), 2)
    for j in range(run_len):
      bitstream.append(f'0b{curr_bit}')
  elif mode == "huffman":
    huffman_tree = decode_huffman_tree(rle_bitstream, rle_bitstream.read(f'uint{METADATA_BITS}'))
    curr_bit = 1 if rle_bitstream.read(1) == '0b1' else 0
    while rle_bitstream.pos != rle_bitstream.length:
      run_len = read_huffman_tree(huffman_tree, rle_bitstream)
      for j in range(run_len):
        bitstream.append(f'0b{curr_bit}')
      curr_bit = 1 - curr_bit
  return bitstream

In [148]:
def compress_matrix(kmeans_data, output_filename, delta_matrix_mode="normal", huffman_mode="vals_only", rle_mode=None):
  reps, labels, delta_matrix = kmeans_data
  n_clusters = len(reps)
  bitstream = BitStream()

  # Compress metadata values
  bitstream.append(f'uint{SIZE_BITS}={delta_matrix.shape[0]}')
  bitstream.append(f'uint{SIZE_BITS}={delta_matrix.shape[1]}')
  bitstream.append(f'uint{METADATA_BITS}={n_clusters}')

  bits_per_m_index = delta_matrix.shape[0].bit_length()
  bits_per_n_index = delta_matrix.shape[1].bit_length()
  bits_per_cluster_index = n_clusters.bit_length()

  # Compress cluster membership information
  for rep in reps:
    bitstream.append(f'bin{bits_per_m_index}={bin(rep)[2:].zfill(bits_per_m_index)}')
  for label in labels:
    bitstream.append(f'bin{bits_per_cluster_index}={bin(label)[2:].zfill(bits_per_cluster_index)}')

  if delta_matrix_mode == "normal":
    compress_delta_matrix(delta_matrix, bitstream, bits_per_n_index)
  elif delta_matrix_mode == "huffman":
    compress_delta_matrix_huffman(delta_matrix, bitstream, bits_per_n_index, mode=huffman_mode)
    
  if rle_mode != None:
    bitstream = encode_rle(bitstream, mode=rle_mode)

  with open(output_filename, 'wb') as f:
    f.write(bytes(bitstream))

def decompress_matrix(input_filename, delta_matrix_mode="normal", huffman_mode="vals_only", rle_mode=None):
  with open(input_filename, 'rb') as f:
    bitstream = BitStream(f.read())

  if rle_mode != None:
    bitstream = decode_rle(bitstream, mode=rle_mode)

  # Decompress metadata
  m = bitstream.read(f'uint{SIZE_BITS}')
  n = bitstream.read(f'uint{SIZE_BITS}')
  n_clusters = bitstream.read(f'uint{METADATA_BITS}')

  bits_per_m_index = m.bit_length()
  bits_per_n_index = n.bit_length()
  bits_per_cluster_index = n_clusters.bit_length()
  
  # Decompress cluster membership information
  reps = []
  for i in range(n_clusters):
    reps.append(int(bitstream.read(f'bin{bits_per_m_index}'), 2))

  labels = []
  for i in range(m):
    labels.append(int(bitstream.read(f'bin{bits_per_cluster_index}'), 2))

  # Decompress delta matrix
  matrix = np.zeros((m, n))
  if delta_matrix_mode == "normal":
    decompress_delta_matrix(matrix, bitstream, bits_per_n_index)
  elif delta_matrix_mode == "huffman":
    decompress_delta_matrix_huffman(matrix, bitstream, bits_per_n_index, mode=huffman_mode)

  for i in range(m):
    if reps[labels[i]] == i:
      continue
    matrix[i] += matrix[reps[labels[i]]]
  return matrix

In [ ]:
# dataset = 'humanBloodCells'

for data in ['humanBloodCells']:#['humanKidneyCells', 'humanSkinCells']:
  dataset = data
  matrix, features, barcodes = load_data(dataset)
  print(dataset)
  print_stats(matrix)
  k_vals = [1, 5, 10, 25, 50]
  for k in k_vals:
    reps, labels, delta_matrix = kmeans(matrix, k)
    compress_matrix((reps, labels, delta_matrix), f'./results/{dataset}/normal/{dataset}_normal_{k}.bin')

    reps, labels, delta_matrix = kmeans(matrix.T, k)
    compress_matrix((reps, labels, delta_matrix), f'./results/{dataset}/transpose/{dataset}_transpose_{k}.bin')

  # 
  kmeans_data = kmeans(matrix, 10)
  compress_matrix(kmeans_data, f'./results/{dataset}/huffman_vals_only.bin', delta_matrix_mode="huffman", huffman_mode="vals_only")
  compress_matrix(kmeans_data, f'./results/{dataset}/huffman_vals_indices.bin', delta_matrix_mode="huffman", huffman_mode="vals_indices")

Number of non-zero entries: 11598877
Matrix shape: (38616, 4661)
Sparsity: 6.444208067267334%


In [ ]:
# kmeans_data = kmeans(matrix, 10)
# compress_matrix(kmeans_data, f'./results/{dataset}/huffman_vals_only.bin', delta_matrix_mode="huffman", huffman_mode="vals_only")
# compress_matrix(kmeans_data, f'./results/{dataset}/huffman_vals_indices.bin', delta_matrix_mode="huffman", huffman_mode="vals_indices")

In [121]:
decompressed_vals_only = decompress_matrix(f'./results/{dataset}/huffman_vals_only.bin', delta_matrix_mode="huffman", huffman_mode="vals_only")
decompressed_vals_indices = decompress_matrix(f'./results/{dataset}/huffman_vals_indices.bin', delta_matrix_mode="huffman", huffman_mode="vals_indices")
print(np.array_equal(matrix, decompressed_vals_only))
print(np.array_equal(matrix, decompressed_vals_indices))

True
True


In [191]:
dataset = 'humanKidneyCells'
with open(f'./results/{dataset}/huffman_vals_only.bin', 'rb') as f:
  bitstream = BitStream(f.read())
initial_bit, run_lengths = get_run_lengths(bitstream)
np.save(f'./results/{dataset}/huffman_vals_only_run_lengths.npy', np.array(run_lengths))
rle_elias_bitstream = encode_rle(None, mode="elias", runs=(initial_bit, run_lengths))
with open(f'./results/{dataset}/rle_elias.bin', 'wb') as f:
  f.write(bytes(rle_elias_bitstream))
rle_huffman_bitstream = encode_rle(None, mode="huffman", runs=(initial_bit, run_lengths))
with open(f'./results/{dataset}/rle_huffman.bin', 'wb') as f:
  f.write(bytes(rle_huffman_bitstream))

Huffman codebook: {17: '1111100001110001010', 1: '0', 3: '110', 2: '10', 19: '11111000011100010010', 4: '1110', 5: '11110', 7: '1111101', 6: '111111', 8: '11111001', 10: '1111100000', 11: '11111000010', 15: '11111000011101', 23: '111110000111000101111', 14: '111110000111001', 12: '111110000110', 13: '1111100001111', 18: '11111000011100001', 9: '111110001', 16: '11111000011100011', 26: '111110000111000000', 20: '111110000111000001111', 24: '1111100001110000010', 27: '11111000011100010110', 32: '111110000111000001100100', 43: '11111000011100000110010110', 22: '111110000111000101110', 28: '11111000011100010011', 25: '1111100001110001000', 21: '111110000111000001101', 29: '111110000111000001110', 30: '1111100001110000011000', 31: '11111000011100000110011', 34: '1111100001110000011001010', 37: '11111000011100000110010111'}
